In [ ]:
import pandas as pd
import os

def filter_ground_truth_by_subject_time_range(
    ground_truth_csv_path,
    synchronized_time_excel_path,
    subject_id,
    output_csv_path
):
    """
    Filters a ground truth CSV file to keep only entries within a specific
    start and end time range for a given subject, based on an Excel file.

    Args:
        ground_truth_csv_path (str): Path to the ground_truth.csv file.
        synchronized_time_excel_path (str): Path to the Synchronized_Time.xlsx file.
        subject_id (str): The subject ID (e.g., 'tr01') to filter by.
        output_csv_path (str): Path to save the filtered CSV file.
    """
    if not os.path.exists(ground_truth_csv_path):
        print(f"Error: ground_truth.csv not found at '{ground_truth_csv_path}'")
        return
    if not os.path.exists(synchronized_time_excel_path):
        print(f"Error: Synchronized_Time.xlsx not found at '{synchronized_time_excel_path}'")
        return

    print(f"Loading ground truth from: {ground_truth_csv_path}")
    try:
        df_gt = pd.read_csv(ground_truth_csv_path)
    except Exception as e:
        print(f"Error reading ground_truth.csv: {e}")
        return

    print(f"Loading synchronized times from: {synchronized_time_excel_path}")
    try:
        df_sync = pd.read_excel(synchronized_time_excel_path)
    except Exception as e:
        print(f"Error reading Synchronized_Time.xlsx: {e}")
        return

    # --- Step 1: Parse Ground Truth Timestamps ---
    # Convert 'Time' column in ground_truth to datetime objects
    # Handle potential BOM in the header if it causes issues ("Time\ufeff")
    if '\ufeffTime' in df_gt.columns:
        df_gt.rename(columns={'\ufeffTime': 'Time'}, inplace=True)

    try:
        # Infer format to handle various potential date/time separators and milliseconds
        df_gt['Time'] = pd.to_datetime(df_gt['Time'], infer_datetime_format=True)
    except Exception as e:
        print(f"Error converting 'Time' column in ground_truth.csv to datetime: {e}")
        print("Please check the 'Time' column format. Expected format: 'DD/MM/YYYY HH:MM:SS.ms'")
        return

    # Extract the date component from the first ground truth timestamp
    # We assume the ground_truth.csv corresponds to a single day's recording session.
    ground_truth_date = df_gt['Time'].iloc[0].date()
    print(f"Ground truth data date identified as: {ground_truth_date}")

    # --- Step 2: Extract Subject-Specific Time Range from Excel ---
    subject_row = df_sync[df_sync['Subject'] == subject_id]

    if subject_row.empty:
        print(f"Error: Subject '{subject_id}' not found in Synchronized_Time.xlsx.")
        print(f"Available subjects: {df_sync['Subject'].tolist()}")
        return

    # Extract start and end times as strings
    start_time_str = str(subject_row['FTP_StartTime'].iloc[0]).strip()
    end_time_str = str(subject_row['FTP_EndTime'].iloc[0]).strip()

    # Combine the ground truth date with the time strings from Excel
    try:
        start_datetime = pd.to_datetime(f"{ground_truth_date} {start_time_str}")
        end_datetime = pd.to_datetime(f"{ground_truth_date} {end_time_str}")
    except Exception as e:
        print(f"Error combining date with Excel times for subject '{subject_id}': {e}")
        print(f"Ground truth date: {ground_truth_date}")
        print(f"Excel Start Time: '{start_time_str}', Excel End Time: '{end_time_str}'")
        print("Please ensure Excel times are in HH:MM:SS format.")
        return

    print(f"Filtering for subject '{subject_id}' between {start_datetime} and {end_datetime}")

    # --- Step 3: Filter Ground Truth Data ---
    df_filtered = df_gt[
        (df_gt['Time'] >= start_datetime) &
        (df_gt['Time'] <= end_datetime)
    ].copy() # Use .copy() to avoid SettingWithCopyWarning

    # --- Step 4: Save Filtered Data ---
    if not df_filtered.empty:
        df_filtered.to_csv(output_csv_path, index=False)
        print(f"Filtered data saved to: '{output_csv_path}'")
        print(f"Original rows: {len(df_gt)}, Filtered rows: {len(df_filtered)}")
    else:
        print(f"No data found for subject '{subject_id}' within the specified time range. No file saved.")

# --- Main execution block ---
if __name__ == "__main__":
    # Define your file paths
    GROUND_TRUTH_CSV = "ground_truth.csv" # Ensure this file is in the same directory or provide full path
    SYNCHRONIZED_TIME_EXCEL = "Synchronized_Time.xlsx" # Ensure this file is in the same directory or provide full path
    
    # --- IMPORTANT: Change this to the subject ID you want to filter for ---
    TARGET_SUBJECT_ID = "tr01" # Example: 'tr01', 'tr03', etc.

    OUTPUT_CSV_FILE = f"ground_truth_filtered_{TARGET_SUBJECT_ID}.csv"

    filter_ground_truth_by_subject_time_range(
        ground_truth_csv_path=GROUND_TRUTH_CSV,
        synchronized_time_excel_path=SYNCHRONIZED_TIME_EXCEL,
        subject_id=TARGET_SUBJECT_ID,
        output_csv_path=OUTPUT_CSV_FILE
    )

    print("\n--- Script finished ---")
    print("If you need to filter for another subject, update TARGET_SUBJECT_ID and re-run the script.")